In [1]:
import math
import numpy as np
from qiskit.circuit import Gate,QuantumCircuit, Parameter, ParameterVector

In [2]:
def binary_conversion(state):
    """
    Convert the index of each non-zero element in 'state' to binary.
    
    Args:
        state (np.array) : An array representing a quantum state.
                            Non zero elements are converted to binary and stored in 'binary_list'.
        
    Returns:
        tuple:            A tuple containing a list of binary representations ('binary_list') and 
                            a list of corresponding non-zero elements ('pro_list') in 'state'. 
    """
    # List to store binary representations
    binary_list = []
    # List to store non-zero elements
    pro_list = []

    # Flatten multi-dimensional array into one dimension
    state = state.flatten()
    
    num_qubit = int(math.log(len(state),2))

    for i in range(len(state)):
        element = state[i]
        # Check if absolute value of element is greater or equal to 1e-6
        if abs(element) >= 1e-6:
            binary_i = bin(np.array(i))[2:]
            # Add leading zeros to 'binary_i' until its length equals 'num_qubit'
            binary_i = binary_i.zfill(num_qubit)
            # Append binary representation and element to respective lists
            binary_list.append(binary_i)
            pro_list.append(element)
    return binary_list, pro_list


In [3]:
import numpy as np
from qiskit.circuit import QuantumCircuit, ParameterVector

def double_circuit(circ, params_list, qubits_list, params_idx):
    """
    Apply a sequence of gates to the circuit (two-qubit operation)
    """

    # Apply gates
    circ.cx(qubits_list[2],qubits_list[3])
    circ.cx(qubits_list[0],qubits_list[2])
    circ.h(qubits_list[3])
    circ.h(qubits_list[0])
    circ.cx(qubits_list[2],qubits_list[3])
    circ.cx(qubits_list[0],qubits_list[1])
    circ.ry(params_list[params_idx]/8, qubits_list[1])
    circ.ry(-params_list[params_idx]/8, qubits_list[0])
    circ.cx(qubits_list[0],qubits_list[3])

    circ.h(qubits_list[3])
    circ.cx(qubits_list[3],qubits_list[1])
    circ.ry(params_list[params_idx]/8, qubits_list[1])
    circ.ry(-params_list[params_idx]/8, qubits_list[0])
    circ.cx(qubits_list[2],qubits_list[1])
    circ.cx(qubits_list[2],qubits_list[0])
    circ.ry(-params_list[params_idx]/8, qubits_list[1])
    circ.ry(params_list[params_idx]/8, qubits_list[0])
    circ.cx(qubits_list[3],qubits_list[1])

    circ.h(qubits_list[3])
    circ.cx(qubits_list[0],qubits_list[3])
    circ.ry(-params_list[params_idx]/8, qubits_list[1])
    circ.ry(params_list[params_idx]/8, qubits_list[0])
    circ.cx(qubits_list[0],qubits_list[1])
    circ.cx(qubits_list[2],qubits_list[0])
    circ.h(qubits_list[0])
    circ.h(qubits_list[3])
    circ.cx(qubits_list[0],qubits_list[2])
    circ.cx(qubits_list[2],qubits_list[3])


def single_circuit(circ, params_list, qubits_list, params_idx, doubles_list):
    """
    Apply a sequence of gates to the circuit (single-qubit operation)
    """

    circ.cx(qubits_list[0],qubits_list[1])
    circ.ry(params_list[params_idx+len(doubles_list)]/2, qubits_list[0])
    circ.cx(qubits_list[1],qubits_list[0])
    circ.ry(-params_list[params_idx+len(doubles_list)]/2, qubits_list[0])
    circ.cx(qubits_list[1],qubits_list[0])
    circ.cx(qubits_list[0],qubits_list[1])


def Adapt_Givens_Ansatz(params_list):
    """
    Create an Adaptive Givens Ansatz circuit
    """

    singles_list = [[6,10]]
    doubles_list = [[1, 2, 10, 11]]

    qubits_num = 12
    circ = QuantumCircuit(qubits_num)
    initial_list = np.array([0,1,2,3,4,6,7,8,9])

    circ.x(initial_list)

    # Apply two-qubit operations
    if doubles_list:
        for i, excitation in enumerate(doubles_list):
            double_circuit(circ=circ, params_list=params_list, qubits_list=excitation, params_idx=i)

    # Apply single-qubit operations
    if singles_list:
        for i, excitation in enumerate(singles_list):
            single_circuit(circ=circ, params_list=params_list, qubits_list=excitation, params_idx=i, doubles_list=doubles_list)

    return circ


In [4]:
from qiskit import QuantumCircuit, transpile, assemble
from qiskit.providers.aer import StatevectorSimulator
import numpy as np

# List of noise words and parameters pool
noise_word = ['fakecairo','fakekolkata','fakemontreal']
params_list_pool = [[0.0160047172520491, 0.0038410454658830604],[-0.11511933211321768, -0.012391014094485356],[0.016699986161650287, -0.0007964624944985476]]

for index, params_list in enumerate(params_list_pool):
    # Create quantum circuit
    circ = Adapt_Givens_Ansatz(params_list)

    # Initialize simulator
    simulator = StatevectorSimulator()

    # Transpile and assemble the circuit for execution
    sim_circuit = transpile(circ, simulator)
    qobj = assemble(sim_circuit)

    # Execute the circuit and get results
    result = simulator.run(qobj).result()

    # Get the state vector from the result
    statevector = result.get_statevector()

    # Convert statevector to_numpy array
    numpy_array = np.array(statevector)

    # Display noise model and ground state
    print(f'The noise model is {noise_word[index]}')
    print(f'The ground state is {binary_conversion(numpy_array)}')


The noise model is fakecairo
The ground state is (['001111011111', '011110011111', '111111011001'], [(0.9999661371549096-3.061513322806258e-16j), (-0.0019204600597247851+5.879713162580732e-19j), (-0.008002273217466396+2.449989570418203e-18j)])
The noise model is fakekolkata
The ground state is (['001111011111', '011110011111', '111111011001'], [(0.998324739426687-3.0564879916212694e-16j), (0.006185207097054466-1.893673518367811e-18j), (0.05752788768766578-1.761283587960208e-17j)])
The noise model is fakemontreal
The ground state is (['001111011111', '011110011111', '111111011001'], [(0.9999650597190275-3.0615100241102484e-16j), (0.0003982173539884511-1.2191890198172133e-19j), (-0.00834989605092545+2.5564183679947436e-18j)])


/tmp/ipykernel_2847669/2539181801.py:21: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  result = simulator.run(qobj).result()
